### **Introduction:** 

In the study RUNX1 contributes to higher-order chromatin organization and gene regulation in breast cancer cells (Barutcu et al., 2016), they looked at how the transcription factor RUNX1 affects gene activity and genome organization in MCF-7 breast cancer cells. RUNX1 can act as both an oncogene and a tumor suppressor, and is known to influence chromatin structure and gene regulation as well. The purpose of this study was to determine how knockdown of RUNX1 affects gene expression and the three-dimensional organization of chromatin in breast cancer cells. To explore this, the authors used RNA-seq to measure which genes are turned on or off when RUNX1 is reduced, ChIP-seq to see where RUNX1 binds across the genome, and Hi-C to examine changes in the three-dimensional structure of the genome including chromatin structure. RNA-seq revealed which genes and noncoding RNAs are affected and ChIP-seq helped identify RUNX1 binding at promoters, enhancers, and other genomic regions. Hi-C further showed that loss of RUNX1 alters chromatin contacts across the genome, indicating changes in overall genome organization. By combining these approaches, the study was able to connect RUNX1 binding with changes in gene expression and genome structure in breast cancer cells.   

### **Methods:** 

The ChIP-seq data analyzed in this project originates from the Barutcu et al., 2016 study, including two biological replicates from human cell lines. Each replicate consists of paired IP (Immunoprecipitate) and INPUT control samples. The IP samples represent DNA sequences enriched for binding of RUNX1 and the INPUT samples serve as controls to account for background signals. This resulted in 4 FASTQ files for downstream analysis.

Initial quality control of the FASTQ files was performed using FastQC v0.12.1 with the parameters -t and --outdir specified, and all other parameters left at default. Adapter sequences and low-quality bases were trimmed from the FASTQ files using Trimmomatic v0.39 with the parameters -threads, ILLUMINACLIP, LEADING, TRAILING, SLIDINGWINDOW, and MINLEN, and all other parameters were left at default. An index was generated for the human reference genome (hg38) using Bowtie2 v2.5.0 with the parameter --threads, and all other parameters left at default. Trimmed reads were then aligned to the reference genome using Bowtie2 with the parameters -p, -x, and -U specified, and all other parameters left at default.

Aligned reads were converted to BAM format and sorted using Samtools v1.17. Sorted BAM files were then indexed using Samtools index, and alignment statistics were obtained using Samtools flagstat. For all Samtools modules, parameters were left at default. Quality control metrics from FastQC, Trimmomatic, and Samtools flagstat were aggregated using MultiQC v1.25 with the parameter -f specified and all other parameters left at default.

For each BAM file, coverage tracks in bigWig format were generated using Deeptools v3.5.5 bamCoverage with the parameters -b and -o specified, and all other parameters left at default. Correlation between bigWig coverage tracks was assessed using Deeptools multiBigwigSummary with the parameters -b, -o, -binSize, and -outRawCounts specified, and all other parameters left at default. Correlation plots were generated using Deeptools with the parameters -in, --corMethod, --whatToPlot, --plotNumbers, --plotTitle, --plotFile, --removeOutliers, and --colorMap specified, and all other parameters left at default. Pearson correlation was used for this plot to capture linear relationships between read counts across genomic bins.

Peak calling was performed using HOMER v4.11. Each sorted BAM file was first converted to a tag directory using makeTagDirectory with default parameters. Peaks were identified using findPeaks using the -style and -i parameters, and all other parameters left at default. Peak outputs were converted to BED format using pos2bed.pl with all other parameters left at default.

A single set of reproducible peaks was generated by intersecting replicate peak sets using Bedtools v2.31.1 intersect with the -a, -b, and -f parameters specified, and all other parameters left at default. A minimum of 10% overlap between peaks was required to consider them reproducible. This threshold ensures that the retained peaks represent a substantial shared signal between replicates while minimizing inclusion of minor overlaps. Peaks overlapping ENCODE blacklist regions by at least 1 bp were removed using bedtools intersect with the -v, -a, and -b parameters specified, and all other parameters left at default. Filtered, reproducible peaks were annotated to their nearest genomic feature using HOMER annotatePeaks.pl with the -gtf parameter, and all other parameters left at default.

Transcription start site (TSS) and transcription termination site (TTS) coordinates for all hg38 genes were obtained from the UCSC Table Browser in BED format. Signal coverage across all gene bodies was computed using Deeptools computeMatrix scale-regions with the -S, -R, -a, --skipZeros, and -o parameters specified, and all other parameters left at default. The resulting matrix was visualized using plotProfile with the -m, --outFileName, --plotTitle, and --perGroup parameters specified, and all other parameters left at default.
Motif enrichment analysis was performed using HOMER findMotifsGenome.pl on the reproducible and filtered peak set, using a 200 bp window (-size 200) and the -mask and -preparsedDir parameters, and all other parameters left at default.

All Nextflow processes were executed on a shared computing cluster using Singularity containers (ghcr.io/bf528/fastqc:latest, ghcr.io/bf528/multiqc:latest, ghcr.io/bf528/bowtie2:latest, ghcr.io/bf528/deeptools:latest, ghcr.io/bf528/trimmomatic:latest, ghcr.io/bf528/bedtools:latest, ghcr.io/bf528/samtools:latest, and ghcr.io/bf528/homer_samtools:latest). Computational resources were allocated according to process labels (process_low, process_medium, process_high).

For further downstream analysis, annotated ChIP-seq peaks were integrated with differential expression RNA-seq results obtained from the paper’s GEO dataset (GSE75070). RNA-seq results were then filtered using the significance thresholds reported in the original study, retaining genes with an adjusted p-value (padj) < 0.01 and an absolute log2 fold change > 1. Functional enrichment of annotated peak-associated genes was performed using ENRICHR v3.4, applying an adjusted p-value threshold of < 0.05, and top enriched pathways were visualized using R v4.4.3. All remaining data integration, statistical analyses, and figure generation were conducted in Python v3.10.19.


### **Quality Control Evaluation:** 

The ChIP-seq reads ranged from 29.7 to 30 million across IP samples and 10.9 to 30.1 million across INPUT samples, with 27.8 to 28.2 million IP reads and 10.1 to 28.6 million INPUT reads mapping to the reference genome hg38. Overall, all samples align at over 90% to the reference genome. INPUT rep2 was sequenced to a lower depth but despite this, there was good sequencing depth and efficient alignment. According to FastQC, the Phred quality score was greater than 30 for all bases across all samples. The Per Base Sequence Content showed warnings for both IP samples and failures for both INPUT samples, suggesting slight biases in base composition at the start or end of reads. Additionally, GC content was 46–47% for IP samples and 43% for INPUT samples. For the Sequence Duplication Levels check, both INPUT samples passed but both IP samples failed, indicating a high level of duplicated reads in the IP samples compared to what was expected. This may reflect PCR amplification or high expression of certain transcripts in the IP samples. Overrepresented sequences were only detected in the IP libraries at 2.1% in rep1 and 3.13% in rep2 which was to be expected, and adapter contamination was minimal ranging from 0 to 3.08% for each sample at each position. Trimmomatic shows that in IP samples 95.6% to 96.6% of reads survived, dropping 3.4 to 4.4% and in INPUT samples, 98.6 to 98.9% survived dropping 1.1 to 1.4%. Overall, the experiment appears to be of sufficient quality for downstream analysis despite minor base composition biases and expected duplication in IP samples.

### **Figure 1:** RUNX1 ChIP-seq Signal Coverage Across Human Genes (Rep1)

<img src="/projectnb/bf528/students/agover11/project-3-agover11/results/IP_rep1_signal_coverage.png" width="700"/>



### **Figure 2:** RUNX1 ChIP-seq Signal Coverage Across Human Genes (Rep2)

<img src="/projectnb/bf528/students/agover11/project-3-agover11/results/IP_rep2_signal_coverage.png" width="700"/>

Figure 1 represents the read coverage of RUNX1 (IP_rep1) across genomic regions, spanning from 2000 bp upstream of the transcription start site (TSS) to 2000 bp downstream of the transcription end site (TES). The sharp peak at the TSS indicates strong RUNX1 enrichment near gene promoters, supporting its role in regulating transcription initiation. This confirms that RUNX1 is a transcription factor. The reduced binding across the remaining gene bodies and near the TES suggests RUNX1 primarily functions at promoters rather than throughout the entire gene. Coverage patterns for both replicates (IP_rep1 and IP_rep2) are highly similar, supporting consistent RUNX1 binding enrichment.

### **Figure 3:** Homer Results

<img src="/projectnb/bf528/students/agover11/project-3-agover11/HomerResults.png" width="700"/>

The results from figure 3 show that RUNX family motifs, including RUNX1, RUNX2, and RUNX-AML, are the most strongly enriched. This means the experiment successfully identified the DNA sequences where the RUNX proteins bind. There are also several related motifs enriched from the bZIP family such as Fosl2, Fra1, Fos, and JunB, which indicate that RUNX1 might work together with these proteins to control gene activity. These results suggest a more complex network of interactions, which could be important for understanding how RUNX1 influences gene regulation and cell behavior in breast cancer.

### **Figure 4:** RUNX1 Peak Proximity 

<img src="/projectnb/bf528/students/agover11/project-3-agover11/PeakProx1.png" width="700"/>

Comparison between my analysis (figure 4) and the published Figure 2F shows differences in the numbers of overlapping RUNX1-bound genes near transcription start sites (TSS) among up and down-regulated genes. For example, I found more upregulated and downregulated genes bound by RUNX1 within ±5 kb of the TSS (93 upregulated and 69 downregulated) than reported in the paper (59 upregulated and 48 downregulated). Conversely, I identified less upregulated and downregulated genes not bound within ±5 kb of the TSS (594 upregulated and 397 downregulated) compared to the paper (628 upregulated and 418 downregulated).

For RUNX1 binding across gene bodies within ±20 kb of the TSS, I found less bound genes (113 upregulated and 78 downregulated) than the paper reported (161 upregulated and 120 downregulated), while the number of genes not bound was slightly higher in my analysis (574 upregulated and 388 downregulated) compared to the paper (526 upregulated and 346 downregulated). Despite these numerical differences, the overall proportion of genes bound by RUNX1 remains similarly low in both analyses with under 20% in mine and under 30% in the original paper.

One main reason for the discrepancies is from the use of hg38 in my analysis instead of hg19, which was used in the paper. This changes gene coordinates and can affect which peaks are linked to genes. Another reason for discrepancies is differences in HOMER versions, which may slightly alter peak annotation or assignment. Together, these factors can influence the total number of genes identified as bound by RUNX1, even when my analysis and the paper used similar peak-calling methods. Combining ChIP-seq peak data with RNA-seq results allows identification of direct RUNX1 targets by linking binding events to gene expression changes. This approach helps clarify how RUNX1’s binding relates to the changes in gene activity that were observed.

### **Figure 5:** ChIP-seq coverage at MALAT1 locus on chr11

<img src="MALAT1.png" width="900"/>


### **Figure 6:** ChIP-seq coverage at NEAT1 locus on chr11

<img src="NEAT1.png" width="900"/>

From my annotated peaks (figures 5 and 6) created using the Integrative Genomics Viewer genome browser, I observe a statistically significant peak overlapping MALAT1 but no significant peak at NEAT1 unlike published figure 2D and 2E from the paper, which shows clear peaks at both genes. My genomic coverage tracks for RUNX1 (IP) and INPUT samples visually resemble those in the paper, displaying signal enrichment over both gene regions and indicating similar binding patterns. Discrepancies between my analysis and the paper likely arise from differences in filtering criteria, such as the minimum 10% peak overlap required to define reproducible peaks and the 1 bp threshold for removing blacklisted regions in my analysis. Another reason for discrepancies is minor differences in genome annotation and how peaks were linked to nearby genes.

### **Figure 7:** 

<img src="/projectnb/bf528/students/agover11/project-3-agover11/Table1.png" width="1000"/>

The raw reads observed in my analysis (figure 7) match those reported in the paper's figure S2A. However, my mapped reads were higher for both RUNX1 (IP) and INPUT samples compared to the paper. One reason for these differences could be from the use of the hg38 reference genome in my analysis instead of hg19 used in the paper, which can affect how many reads successfully align. Additionally, slight differences in Bowtie2 versions or alignment parameters, as well as differences in adapter trimming parameters, could contribute to the increased number of mapped reads in my results.

### **Figure 8:** Correlation Plot

<img src="/projectnb/bf528/students/agover11/project-3-agover11/results/correlation_plot.png" width="700"/>

I chose Pearson correlation for this plot because it captures linear relationships between read counts across genomic bins. This is appropriate for comparing IP and INPUT samples, which are expected to show similar coverage at the same genomic locations. Spearman correlation would be less sensitive to differences in signal intensity, while Pearson better reflects biological similarity in our ChIP-seq data.

My analysis (figure 8) shows significantly lower correlations compared to the paper's figure S2B. For example, my IP replicates correlate at 0.64 versus the paper's 0.91, and INPUT replicates at 0.89 versus 0.95. In the paper, the authors reported high reproducibility among their RUNX1 ChIP-seq replicates, which supports their claim of successfully identifying 3,466 reproducible RUNX1 ChIP-seq peaks. However, my IP replicate correlation of 0.64 is not as high as expected, and the INPUT-to-IP correlations range from 0.49-0.64, suggesting potential issues with enrichment quality or technical variability. While samples cluster correctly, the results from my analysis indicate that my experiment may have quality issues and further investigation of peak reproducibility and enrichment scores is needed before I can conclude it was successful.

### **Figure 9:** Venn diagram

<img src="/projectnb/bf528/students/agover11/project-3-agover11/VennPeaks.png" width="700"/>

The Venn diagram (figure 9) comparing peaks between replicates shows significant differences between my analysis and the paper’s results in figure S2C. I observed 82,898 unique peaks in replicate 1, 16,435 unique peaks in replicate 2, and 6,462 overlapping peaks, whereas the paper reported 3,983 unique peaks in replicate 1, 10,465 in replicate 2, and 3,466 overlapping peaks. These discrepancies can likely be attributed to my use of the hg38 reference genome, which contains updated and additional genomic regions compared to hg19. Using the hg38 reference genome potentially results in more peaks being identified overall. Higher mapped read counts in my analysis may also have increased signal coverage allowing HOMER to detect more peaks. Additionally, minor variations between HOMER versions could also contribute to differences in peak detection. These factors explain why the number of called peaks and overlaps is substantially higher in my analysis than in the paper’s results. 

### **Figure 10:** Top Enriched Pathways

<img src="EnrichR_top.png" width="1000"/>

Functional enrichment was performed on the annotated peaks list using ENRICHR v3.4 as seen in figure 10. The gene lists was run against the GO Biological Process, KEGG, and Reactome databases, ranking enriched terms by their adjusted p-value. Terms with an adjusted p-value < 0.05 were retained for further analysis. Enrichment pathways were then visualized in R v4.4.3 by plotting the top pathways according to –log10 (adjusted p-value). The top results included Translocation of SLC2A4 (GLUT4) to Plasma Membrane, RNA binding, cadherin binding, and Chromatin modifying enzymes. These results indicate that RUNX1 plays a supporting role in chromatin regulation, RNA-associated processes, and cell adhesion dynamics during transcriptional control and cellular differentiation. However, the enrichment of the GLUT4 translocation pathway was surprising since RUNX1 is not typically associated with GLUT4. This could be explained by RUNX1 indirectly influencing genes involved in pathways related to metabolism or protein transport where GLUT4 plays a role.
